In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
from ilqr.cost import QRCost, FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'ilqr.cost'

In [5]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt, final_state)

In [6]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'inverted_double_pendulum.xml')
dynamics = MujocoDynamics(xml_path, frame_skip = 4, use_multiprocessing = True)
print(dynamics.dt)

Finished loading processFinished loading processFinished loading process  30469 3047130470
Finished loading process

 30475
Finished loading process Finished loading processFinished loading processFinished loading process  30486 3050630514

30474

Finished loading processFinished loading process  30534Finished loading process30521

 30543
Finished loading process 0.04
30554


In [7]:
print(dynamics.state_size)
x_goal = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Instantenous state cost.
Q = np.eye(dynamics.state_size)
Q[0, 0] = 5.0
Q[1, 1] = 50.0
Q[2, 2] = 50.0
Q[3, 3] = 20.0
Q[4, 4] = 700.0
Q[5, 5] = 700.0



# Terminal state cost.
Q_terminal = 10 * Q

# Instantaneous control cost.
R = np.eye(1)

cost1 = QRCost(Q, R, Q_terminal=Q_terminal, x_goal=x_goal)

6


In [8]:
def l(x, u, i):
    c0 = x[0] ** 2
    c1 = 10 * ((np.cos(x[1]) - 1) ** 2)
    c2 = 10 * ((np.cos(x[2]) - 1) ** 2)
    c3 = x[3] ** 2
    c4 = x[4] ** 2
    c5 = x[5] ** 2
    cu = 0.5 * u[0] ** 2
    return c0 + c1 + c2 + c3 + c4 + c5 + cu

cost2 = FiniteDiffCost(l, lambda x, i: l(x, [0.0], i), 6, 1, use_multiprocessing = True)

Finished loading process Finished loading process 3058930586
Finished loading process
 30596Finished loading process 30611

Finished loading process 30614
Finished loading processFinished loading process  30629
30634Finished loading processFinished loading process 
 3063930650

Finished loading processFinished loading process  3066730670

Finished loading process 30675


In [9]:
N = 100
x0 = np.array([0.0, np.random.uniform(-np.pi, np.pi), np.random.uniform(-np.pi, np.pi), 0.0, 0.0, 0.0])
#x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
#us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N)
mpc = RecedingHorizonController(x0, ilqr)

In [10]:
t0 = time.time()
J_hist = []
controls = mpc.control(us_init, step_size = 3, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []
for i in range(30):
    print('ITERATION', i, '\n')
    if i == 29:
        us.append(next(controls)[2])
    else:
        us.append(next(controls)[1])
    
print('time', time.time() - t0)
us = np.concatenate(us)

ITERATION 0 

iteration 0 accepted 21504.762300151317 [-2.72781373  5.71863428 14.20556025  3.28354218 -5.87467738 -5.88304373]
iteration 1 accepted 21107.889730684234 [-2.77013635  5.7059985  14.23415778  2.99544122 -5.85359287 -5.6969541 ]
iteration 2 accepted 20781.84037932826 [-2.76845238  5.73041809 14.2359873   2.89753025 -5.69309456 -5.35502614]
iteration 3 accepted 20434.73961805257 [-2.76092877  5.76008279 14.22633826  2.82418032 -5.50390737 -4.99070625]
iteration 4 accepted 20335.21416296868 [-2.75819932  5.7714576  14.21654802  2.81487415 -5.44401233 -4.90242647]
iteration 5 accepted 20214.1830478717 [-2.76272322  5.77950744 14.18958919  2.79536106 -5.39273614 -4.89455357]
iteration 6 accepted 19777.34193771102 [-2.69698581  5.9911739  14.07557783  3.01289972 -4.93507543 -4.1182529 ]
iteration 7 accepted 19566.93916004855 [-2.70405329  6.03969041 14.05564292  2.88550269 -4.72567398 -3.8408592 ]
iteration 8 accepted 19347.716555753796 [-2.7104192   6.08410355 14.05908367  2.7

iteration 15 failed 17823.15459132138 [-2.42342307  8.56993207 14.16824753  4.45306954  6.12582234 -4.64311456]
iteration 16 accepted 17822.688173186732 [-2.42415559  8.56992398 14.16779205  4.45220875  6.12623961 -4.64302266]
iteration 17 failed 17822.68817318673 [-2.42415559  8.56992398 14.16779205  4.45220875  6.12623961 -4.64302266]
iteration 18 failed 17822.68817318673 [-2.42415559  8.56992398 14.16779205  4.45220875  6.12623961 -4.64302266]
iteration 19 accepted 17818.287367391655 [-2.43893289  8.56583506 14.15081198  4.43902542  6.11927983 -4.60512558]
iteration 20 accepted 17816.74622886547 [-2.43547044  8.56940032 14.14623737  4.44573391  6.1286201  -4.64875212]
iteration 21 failed 17816.746228865464 [-2.43547044  8.56940032 14.14623737  4.44573391  6.1286201  -4.64875212]
iteration 22 accepted 17816.7045419932 [-2.4357472   8.56948552 14.14576547  4.44551502  6.12906451 -4.65011279]
iteration 23 failed 17816.704541993196 [-2.4357472   8.56948552 14.14576547  4.44551502  6.129

iteration 16 accepted 16049.82150200132 [-2.68502724  8.07415082 13.66730311  2.40503116  3.98970193 -2.29890799]
iteration 17 accepted 16007.407325565515 [-2.7132123   8.04691755 13.89872516  2.34788369  3.96164415 -1.93907551]
iteration 18 accepted 15777.621849848716 [-2.5563597   8.1708925  13.65536695  3.05045447  4.14896967 -2.06392118]
iteration 19 accepted 15771.012308748977 [-2.55671174  8.16690263 13.65656043  3.03824329  4.1285918  -2.05467605]
iteration 20 accepted 15766.326532388704 [-2.557168    8.16300577 13.65778897  3.02611623  4.1088792  -2.04545021]
iteration 21 accepted 15758.577830081615 [-2.63863264  8.02155944 13.75414294  2.44350411  3.48832647 -1.79783499]
iteration 22 accepted 15734.672613182984 [-2.64417434  8.02275319 13.75655082  2.42999098  3.49328543 -1.76930433]
iteration 23 accepted 15730.236404187799 [-2.64711321  8.02374237 13.7571597   2.42228345  3.49742173 -1.75974866]
iteration 24 accepted 15718.847892563064 [-2.67078548  8.0655533  13.73674182  2.

iteration 89 accepted 11877.630843472723 [-0.10999559  7.33664391 12.23363301  0.67294709  3.07466576 -3.08562373]
iteration 90 accepted 11861.51981945041 [-0.1550159   7.18973869 12.36469395  1.26144494  2.11021736 -2.60559827]
iteration 91 accepted 11841.780812633493 [-0.18145604  7.09883266 12.42140693  1.60319178  1.29067848 -1.81906224]
iteration 92 accepted 11825.118291508274 [-0.17296226  6.99542966 12.46641689  1.66278107  0.62785433 -1.0891406 ]
iteration 93 accepted 11815.399441275695 [-0.18037514  6.98007653 12.46555352  1.67273085  0.53567622 -1.04068263]
iteration 94 accepted 11797.919805410032 [-0.176458    6.92691556 12.46415229  1.68323533  0.20790426 -0.82315266]
iteration 95 accepted 11795.864733247448 [-0.21191781  6.92806062 12.45526937  1.7333288   0.15692739 -0.87333364]
iteration 96 accepted 11787.322642150379 [-0.22478165  6.92430838 12.4493278   1.72772132  0.13128068 -0.89699172]
iteration 97 accepted 11782.792947294976 [-0.17691197  6.92411789 12.44370068  1.

iteration 62 accepted 10745.190075443312 [-0.18798263  7.53342201 10.57916905 -0.4520475   1.3901555  -2.33644527]
iteration 63 accepted 10737.09082717889 [-0.02205174  7.67098119 10.59834647 -0.54521087  1.8475527  -1.53885923]
iteration 64 accepted 10703.524007719483 [-0.10373899  7.70047741 10.46466255 -0.68883543  1.76050205 -1.3143398 ]
iteration 65 accepted 10671.691351117463 [-0.27119348  7.69028836 10.03788134 -1.19187401  1.34936975 -1.2200336 ]
iteration 66 accepted 10650.99010811035 [-0.32662651  7.74301926 10.37214771 -0.8347117   1.32139094 -1.16760283]
iteration 67 accepted 10616.844596186424 [-0.28058112  7.79453589 10.12735429 -0.78426317  1.20737708 -1.04580603]
iteration 68 failed 10616.844596186418 [-0.28058112  7.79453589 10.12735429 -0.78426317  1.20737708 -1.04580603]
iteration 69 failed 10616.844596186418 [-0.28058112  7.79453589 10.12735429 -0.78426317  1.20737708 -1.04580603]
iteration 70 failed 10616.844596186418 [-0.28058112  7.79453589 10.12735429 -0.7842631

iteration 46 accepted 8074.529989718368 [ 0.63662971 10.32669609  7.05130361  3.08590066 -3.26624543 -1.22284879]
iteration 47 accepted 8029.661288758018 [ 0.89825026 10.22945959  7.16300203  2.65181031 -4.35452376 -1.78374031]
iteration 48 accepted 7940.615321016885 [ 0.69259319 10.34996552  6.94962404  2.96828694 -3.32968521 -1.04895395]
iteration 49 accepted 7872.249254419799 [ 0.39661197 10.19097895  6.75449155  2.17774879 -3.38220409 -1.03720581]
iteration 50 accepted 7848.977219124959 [ 0.17092897 10.30911729  6.6032191   2.45890569 -1.63540273 -0.47530134]
iteration 51 accepted 7844.787802546621 [ 0.15435511 10.28732499  6.56993912  2.17441577 -1.13214056 -0.28516588]
iteration 52 accepted 7841.957171433328 [ 0.32119871 10.14817145  6.3857337   1.81719898  0.19118958  0.52913612]
iteration 53 accepted 7679.0434130529575 [ 0.0353715  10.27038054  6.46064104  1.09588159  0.0837537   0.37619021]
iteration 54 failed 7679.043413052955 [ 0.0353715  10.27038054  6.46064104  1.09588159 

iteration 53 accepted 6308.722865052914 [ 0.29417306  7.99303697  5.83746528 -0.14827014  4.08602242  1.31322625]
iteration 54 accepted 6305.165337932221 [ 0.27702604  7.98866423  5.83303967 -0.15743297  4.07677188  1.29676371]
iteration 55 accepted 6302.752423795236 [ 0.28881745  7.98609896  5.83557429 -0.15405689  4.06315622  1.26916838]
iteration 56 accepted 6297.243260115859 [ 0.28176507  7.98033817  5.83267318 -0.16071335  4.04658563  1.24138579]
iteration 57 accepted 6289.606117485286 [ 0.29192374  7.96829928  5.83227911 -0.1852029   4.02666894  1.18832511]
iteration 58 accepted 6286.8328657257625 [ 0.45346188  7.95806101  5.88210192 -0.20809748  4.02124119  1.05790149]
iteration 59 accepted 6283.983032987477 [ 0.46760641  7.97075691  5.88569753 -0.11084052  3.9622694   1.08439225]
iteration 60 accepted 6278.986372489807 [ 0.4453521   7.96748542  5.87660142 -0.09941299  3.94221332  1.08751455]
iteration 61 accepted 6274.198276453003 [ 0.42165104  7.96396627  5.86774016 -0.0960330

iteration 29 accepted 5740.72174351869 [0.19233126 8.12098744 5.37087926 0.58023616 3.30502858 1.35837157]
iteration 30 accepted 5737.857902969056 [0.19888199 8.11952035 5.36385067 0.60094265 3.27440538 1.34540542]
iteration 31 accepted 5732.047833414416 [0.17286853 8.10916509 5.34919696 0.54601193 3.2588069  1.31607267]
iteration 32 accepted 5724.457332599315 [0.15192025 8.08966409 5.32331594 0.48399278 3.21758465 1.26431183]
iteration 33 accepted 5722.466727398584 [0.2748285  8.09611426 5.30774935 0.77456766 3.05218665 1.24158663]
iteration 34 accepted 5718.738230617781 [0.25455305 8.0937081  5.30157108 0.74440893 3.0481435  1.23198795]
iteration 35 accepted 5715.841818911931 [0.23885283 8.09231961 5.29627739 0.72318054 3.04274744 1.22413244]
iteration 36 accepted 5713.380515603744 [0.22511565 8.09095452 5.29116389 0.70414906 3.03693312 1.21649648]
iteration 37 accepted 5710.820818730975 [0.20950669 8.08831954 5.28532493 0.67815961 3.03224257 1.20681348]
iteration 38 accepted 5708.03

iteration 5 accepted 5149.565050928323 [ 0.63746537  9.68506936  4.85118343  4.56786837 -0.05296149  7.73989994]
iteration 6 accepted 5148.609292034179 [0.79098307 9.75969175 5.01206717 5.97621827 1.063444   8.31760489]
iteration 7 accepted 5123.334611921035 [0.7276856  9.72399404 4.96690294 5.66070245 0.94471128 7.91331411]
iteration 8 accepted 5111.97374662394 [0.68694877 9.71099865 4.95231431 5.54848889 0.91647361 7.78011666]
iteration 9 accepted 5104.462384159259 [0.66976728 9.70260129 4.93611709 5.46636235 0.89768116 7.6623626 ]
iteration 10 accepted 5097.577390788986 [0.65386769 9.69569547 4.9210823  5.39462335 0.88420446 7.55849643]
iteration 11 accepted 5090.930929206301 [0.63867772 9.68966824 4.90618388 5.32717242 0.8728804  7.46108795]
iteration 12 accepted 5084.272419812101 [0.62377879 9.6839703  4.89069662 5.26105295 0.86342456 7.3634405 ]
iteration 13 accepted 5077.2679747923185 [0.60872172 9.6776103  4.8735169  5.18889599 0.8505813  7.25877018]
iteration 14 accepted 5069.

iteration 0 accepted 2604.8751450032555 [-8.80054018e-02  1.25589312e+01 -1.78417104e-03 -5.07605819e-03
 -1.32163497e-02 -5.98465401e-03]
iteration 1 accepted 2604.8709789956 [-8.51298901e-02  1.25593449e+01 -1.20046121e-03 -4.02660694e-03
 -1.03292918e-02 -4.67282199e-03]
iteration 2 converged 2604.868508498685 [-8.34570380e-02  1.25596872e+01 -9.12802430e-04 -3.10261292e-03
 -8.60854298e-03 -4.10087337e-03]
ITERATION 10 

iteration 0 converged 2497.897973095274 [-7.33446474e-02  1.25616048e+01 -6.61635538e-04  1.95575970e-03
 -4.41739066e-03 -2.56352746e-03]
ITERATION 11 

iteration 0 accepted 2405.97479921959 [-6.93795380e-02  1.25603707e+01 -1.92868660e-04 -3.28942372e-03
 -4.81371446e-03 -1.31919514e-03]
iteration 1 accepted 2405.9718864683578 [-6.70868392e-02  1.25610691e+01 -1.54112342e-04 -8.41420824e-04
 -3.31163510e-03 -1.26380618e-03]
iteration 2 converged 2405.970312204899 [-6.57449768e-02  1.25613603e+01 -7.22007899e-05  2.24178427e-04
 -2.42434156e-03 -1.11492589e-03]
IT

In [11]:
dynamics.set_state(x0)
print(dynamics.get_state())
video = []
for i in range(us.shape[0]):
    dynamics.step(us[i])
    video.append(dynamics.sim.render(512, 512))

[0.        1.1013313 2.9453409 0.        0.        0.       ]


In [12]:
print(us.shape)

(187, 1)


In [13]:
make_video_fn(video)()